## Large Language Model (LLM)

### Unleash Language Model's Creativity By Tweaking One Parameter

In [ ]:
!pip install langchain pydotenv

LangChain is an open-sourced Python library that allows you to effortlessly build applications with language models. 

To increase the randomness of the model's output, increase the value of the `temperature` parameter. 

In [1]:
from dotenv import load_dotenv

# Load OPEN_API_KEY from .env
load_dotenv()

True

In [24]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.1)
text = "What would be a good company name that uses drones to deliver everyday items?"
print(llm(text))



DroneDrop Delivery.


In [26]:
# More random output
llm = OpenAI(temperature=0.9)
text = "What would be a good company name that uses drones to deliver everyday items?"
print(llm(text))




FlyByRefill.


[Link to LangChain](https://github.com/hwchase17/langchain).